In [16]:
#Load Package
import pandas as pd

#Read the log file
df = pd.read_csv("Example DAQ Raw data.log", delim_whitespace=True, on_bad_lines='skip', low_memory=False)

#Variables
Gr = 351.2
Gf = 2.04

#Constants
Vex = 2.5
E_Alu = 70
Lja = 201

#Load Cell Conversion Factors
Lcf2T = 38.471
Lcf10T = 192.353
Lcf50T = 961.76

#Calculations
Vo = Vex * Gr * (1 + 1 / 10 ** 6) / (Gr + Gr * (1 + 1 / 10 ** 6))
Vd = Vo - (Vex / 2)
Vdf = Vd * Lja
Vdr = 1 / Vdf

#Join the date columns to create a datetime column
df = df[df['Hour'].notna()]
df['Datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Min', 'Sec']].astype({'Hour': int, 'Min': int, \
                                'Sec': int}).astype(str).agg('-'.join, axis=1), format ='%Y-%m-%d-%H-%M-%S')

#Create the ID column
df['ID'] = df.groupby('Cycles').cumcount() + 1

load_cell = Lcf2T  # <<<<<<<<<<<<<<<<Change your load cell here>>>>>>>>>>>>>>>>>

#Create Load [kN] column
base_load = df['LCell'].iloc[0]
df['Load [kN]'] = (df['LCell'] - base_load) * load_cell

#Create Fcalc Scaled column
base_fcalc = df['FCalc'].iloc[0]
df['Fcalc Scaled'] = (df['FCalc'] - base_fcalc) * 140

#Create Strain columns
for i in range(9):
    base_stress = df['Strain' + str(i)].iloc[0]
    df['SG' + str(i + 1) + ' Stress [Mpa]'] = ((df['Strain' + str(i)] - base_stress) * Vdr / Gf) * E_Alu / 1000
    
#Create RPR Scaled column
base_rpr = df['RPR'].iloc[0]
df['RPR Scaled'] = (df['RPR'] - base_rpr) * 40

#Select relevant columns
final_df = df[['Datetime', 'Cycles', 'ID', 'RamPosR', 'SG1 Stress [Mpa]', 'SG2 Stress [Mpa]', 'SG3 Stress [Mpa]' , \
              'SG4 Stress [Mpa]', 'SG5 Stress [Mpa]', 'SG6 Stress [Mpa]', 'SG7 Stress [Mpa]', 'SG8 Stress [Mpa]' , \
              'SG9 Stress [Mpa]', 'Load [kN]', 'Fcalc Scaled', 'RPR Scaled', 'PLC8' , 'PLC9']]

#Load Data in csv for Power BI utilization the below code needs to be changed so that it can loaded in Dataverse
final_df.to_csv('DAQ Data Conversion.csv', index = False)